# Протоколы обмена ключами

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

In [3]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

In [4]:
def randint(a, b):
    """
    Генерация слуайного числа среди заданных.
    """
    
    native_a = int(a)
    native_b = int(b)
    return gmpy2.mpz(random.randint(native_a, native_b))

----

In [5]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime
from Crypto.Random import random

# HKDF и то, что надо для нее
from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA512

# Подпись и шифрование
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Cipher import AES

# Математическая библиотека
import gmpy2

# Для генерации идентификаторов
import uuid

# Утилитки
from Crypto.Util.number import long_to_bytes
import binascii

----

## Протокол Диффи-Хеллмана
![Diffe-Hellman Protocol](./images/Diffie-Hellman_4.png)

In [6]:
class DiffieHellmanParams(object):
    def __init__(self, prime_bits: int = 512):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[DiffieHellmanParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [7]:
class DiffieHellmanUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: DiffieHellmanParams, 
                 use_another_value: bool = False):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        self._sabotage = use_another_value
        
        trace('[DiffieHellmanUser]', f'client {self._id} created')
    
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, отправляю другому пользователю
        # Получаю от него g ** y mod p
        #
        
        x = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{x = }')
        
        intermediate_value = another_client._accept_key_exchange(gmpy2.powmod(self._g, x, self._p))
        
        #
        # Если требуют неудачную попытку, то подменю x
        # Это как бы незнание изначального значения - попытка угадать
        #
        
        if self._sabotage:
            x = self._generate_group_element()
            trace('[DiffieHellmanUser]', f'[{self._id}]', f'guessed {x = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value) -> int:
        #
        # Получаю g ** x mod p, генерирую у
        #
        
        y = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{y = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return gmpy2.powmod(self._g, y, self._p)
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [8]:
# Параметры протокола
params = DiffieHellmanParams()

[TRACE] [DiffieHellmanParams] p = 11465876652688631126595357184263766434880469303221170115424227414168778980854899156424574734996377220242600003473937083275665369865116251680589485539525179, g = 7251943245509185890191605019803845073923310143108877049342158443072334683562323943468353071199061804687879637134551978430130424380324888187456403163174783


In [9]:
# Два хороших пользователя
alice = DiffieHellmanUser(params)
bob   = DiffieHellmanUser(params)

[TRACE] [DiffieHellmanUser] client 382afa02-c18c-4dad-9909-3b479023f0a3 created
[TRACE] [DiffieHellmanUser] client b96d1e99-d3b8-4e1f-87d6-33ee51263c3c created


In [10]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [382afa02-c18c-4dad-9909-3b479023f0a3] x = mpz(1793505725424406812818904658471012257284438444871665981778706489135202935469927497935378847698581665580365945543514416129728820100630815888568070266437483)
[TRACE] [DiffieHellmanUser] [b96d1e99-d3b8-4e1f-87d6-33ee51263c3c] y = mpz(10676555159610508068926102006620315751180271839537471337522907103103928023804388983615879870325108261645365791925045879770573619674608506240874471348718485)
[TRACE] [DiffieHellmanUser] [b96d1e99-d3b8-4e1f-87d6-33ee51263c3c] derived key: 28c1b7d8c407edea2bcb5bee175eaa99d63eed46c9094553b24120d5e5a886ce
[TRACE] [DiffieHellmanUser] [382afa02-c18c-4dad-9909-3b479023f0a3] derived key: 28c1b7d8c407edea2bcb5bee175eaa99d63eed46c9094553b24120d5e5a886ce


In [11]:
# Теперь врага создам
mallory = DiffieHellmanUser(params, True)

[TRACE] [DiffieHellmanUser] client a6308e86-cb29-436d-8a79-59c1735577c4 created


In [12]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [a6308e86-cb29-436d-8a79-59c1735577c4] x = mpz(6959222405637686139913677142758315494854324311829117394694354694508887393917691279774022175979201597200276139601657171004217087883398751708752192110627372)
[TRACE] [DiffieHellmanUser] [b96d1e99-d3b8-4e1f-87d6-33ee51263c3c] y = mpz(8241549131260886693765024274868898982942914158086354535958174283536554999237686449410481308433907596138237342401806011523316115091820285314640715900065852)
[TRACE] [DiffieHellmanUser] [b96d1e99-d3b8-4e1f-87d6-33ee51263c3c] derived key: e424760d9b1cdca71a05fb14f394a98413b4885bafa8677ae9a1cdc3cea49993
[TRACE] [DiffieHellmanUser] [a6308e86-cb29-436d-8a79-59c1735577c4] guessed x = mpz(4899509630290621772284663112863036999039950235991337953981903341469208175356945349091168097825876928679372914045503520475282905036980579800445165020196061)
[TRACE] [DiffieHellmanUser] [a6308e86-cb29-436d-8a79-59c1735577c4] derived key: 463cb08e107696810f65d185298746bcc06f919f3e8dcf6d2dc57b53535ed630


----
## Протокол MTI
![Diffe-Hellman Protocol](./images/MTI_4.png)

In [13]:
class MTIParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[MTIParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [14]:
class MTIUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: MTIParams, *,
                 fake_data: gmpy2.mpz = None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = self._generate_group_element()
        self._public_key  = gmpy2.powmod(self._g, self._private_key, self._p) if fake_data is None else fake_data
        
        trace('[MTIUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
        
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому клиенту и получаю от него g ** y mod p
        #
        
        intermediate_value = another_client._accept_key_exchange(m, self.public_key)
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(another_client.public_key, x, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
    
    
    def _accept_key_exchange(self, intermediate_value, public_key):
        #
        # Генерирую у, вычисляю g ** y mod p
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(public_key, y, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(MTIUser.HKDF_DUMMY_STUFF, 
                         MTIUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return m
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [15]:
# Параметры протокола
params = MTIParams(512)

[TRACE] [MTIParams] p = 12856360547791008555188228280325485783426208312816825429677348378396159644962061029655070313168212010617984544815140040713734288186553786197638512070747839, g = 3799663994283310145340779494309993717666617460896328315108171617305651451220732653865779966557322940890252177905611945731819427958389496931315814311063480


In [16]:
# Два хороших пользователя
alice = MTIUser(params)
bob   = MTIUser(params)

[TRACE] [MTIUser] client 3ec637b7-1229-439a-9503-5fc86bc65ae8 created. PK = 12393688686660951462079363605631499181923911531168273387867699163769620986299799182264292681742548719820698358412603701716610387952872740503696751122224111, SK = 8664697192275669703314325894699857585097813130791146996237494126222305907809860339554434849897614812318310402246263977447803997925268154282324366235524214
[TRACE] [MTIUser] client c68b5b01-ef2f-47ff-8b87-41f48a0cc8af created. PK = 9808657321559533862412359276330515458924716676408152523063392093651648555578263125088933898856380448445230901862486337816449646273437709793306056999431473, SK = 10627837925704474882641797631710317039905496741685111277616981162818768432233331596103625674098602460245708732329319681353608966581725182744289383643762536


In [17]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [MTIUser] [3ec637b7-1229-439a-9503-5fc86bc65ae8] x = mpz(5183741657946812478600456464521587869391932820586367972952866285613425812395913835058763794950412047988011290974107035436079993136189541857931340284592549), m = mpz(6704747640295379087313488714466537658042959993656145160208849401327254363535593975397272724400424485996868415391332440842911039257420144919071074977057586)
[TRACE] [MTIUser] [c68b5b01-ef2f-47ff-8b87-41f48a0cc8af] y = mpz(12281629934580829556878487695330557313509352177313180187777808752734797794835312505008303470694541395345530614437657268868993802043617778935468248082890400), m = mpz(12546096595808658163714155449514160126806333859545859326243979161819317918050553275485271617060760138543355248436899523296326045110167857802655715875414858)
[TRACE] [MTIUser] [c68b5b01-ef2f-47ff-8b87-41f48a0cc8af] derived key: 39cf0a7444ef16f5c059b6e31fb235bd898a278e90f8a356a15ae50fb149696e
[TRACE] [MTIUser] [3ec637b7-1229-439a-9503-5fc86bc65ae8] derived key: 39cf0a7444ef16f5c059b

In [18]:
# Теперь врага создам
mallory = MTIUser(params, fake_data=alice.public_key)

[TRACE] [MTIUser] client 232f266b-5cbc-420c-b14d-04cba7908699 created. PK = 12393688686660951462079363605631499181923911531168273387867699163769620986299799182264292681742548719820698358412603701716610387952872740503696751122224111, SK = 12524341677475151974042159386777794472468816225124473518898619717396448320942109856248425431042624670546793309706940601419691811116106305347616664658212994


In [19]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [MTIUser] [232f266b-5cbc-420c-b14d-04cba7908699] x = mpz(9934972552373637548053305388018332550333403258060257286844769851791384183425889965110007546023041075097100212017814466479777933782633764917776110865504485), m = mpz(7063221210134045526813098182522519845829464821647199877626639316716790361017327207729948814759937577259290668248980915700786387014679407583603172356408761)
[TRACE] [MTIUser] [c68b5b01-ef2f-47ff-8b87-41f48a0cc8af] y = mpz(7215278394999540961314401275393367402394310577087895935122495494484979436141779285936801948999162877468531570052429962919756688728551012785492564702595844), m = mpz(10341827833186921826113898801716918469065203891775358866635246485122019795570219314096219725763036299575254449370122782038052289379520236997694646459930900)
[TRACE] [MTIUser] [c68b5b01-ef2f-47ff-8b87-41f48a0cc8af] derived key: 854bede16a0bac5f5723e4a712f52d9744d2dd4c29eb7c301ee651250687728c
[TRACE] [MTIUser] [232f266b-5cbc-420c-b14d-04cba7908699] derived key: cf2536fffc879b9383d059

----
## Протокол STS
![STS Protocol](./images/STS_4.png)

In [20]:
class STSParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[STSParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [21]:
class STSUser(object):
    RSA_KEY_LENGTH = 4096
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: STSParams, *, 
                 fake_data=None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = RSA.generate(STSUser.RSA_KEY_LENGTH)
        self._public_key  = self._private_key.publickey() if fake_data is None else fake_data
        
        trace('[STSUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому пользователю, получаю g ** y mod p и зашифрованную подпись
        #
        
        intermediate_value, encrypted_signature = another_client._accept_key_exchange(m)
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Расшифровываю подпись и проверяю ее
        # Если не подошла, то ключ неверный, завершаюсь
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(intermediate_value, m, received_signature, another_client.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return
        
        #
        # Сам теперь подписываю значения и отправляю на проверку другому пользователю
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        if not self._check_key_exchange(m, intermediate_value, 
                                        (cipher.nonce, cipher.encrypt(signature)), self.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Another user refused key exchange')
            self._key = None
            return
        
        #
        # Тут все хорошо
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value):
        #
        # Генерирую у, вычисляю g ** y mod p
        # Вывожу ключ
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Создаю подпись значений и отправляю инициатору
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        encrypted_signature = (cipher.nonce, cipher.encrypt(signature))
        
        return m, encrypted_signature
    
    
    def _check_key_exchange(self, f, s, encrypted_signature, pk):
        #
        # Проверяю подпись от инициатора
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(f, s, received_signature, pk):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return False
        
        #
        # Если все валидно, то ключ верно установлен
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        return True
        
    
    def _sign_values(self, f, s):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        return pss.new(self._private_key).sign(h)
    
    
    def _check_signature(self, f, s, signature, pk):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        verifier = pss.new(pk)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
    
        
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key

-----

In [22]:
# Параметры протокола
params = STSParams(512)

[TRACE] [STSParams] p = 9775134541162501022134590897955420851422423373042933163895453499790773178291095700900761017250327787528404105737871951106208657504039565379300427935315679, g = 9074978892177560017271239829795624720531936610475194481956524754125650327400031949405433747380449974687954959864559310411788434097443539943378384434581188


In [23]:
# Два хороших пользователя
alice = STSUser(params)
bob   = STSUser(params)

[TRACE] [STSUser] client 41a93646-3863-4735-9be3-fbef365c1b69 created. PK = Public RSA key at 0x104697400, SK = Private RSA key at 0x104669460
[TRACE] [STSUser] client c80d165b-9cda-4148-b3bb-273f62780b57 created. PK = Public RSA key at 0x1047DF490, SK = Private RSA key at 0x104438190


In [24]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [STSUser] [41a93646-3863-4735-9be3-fbef365c1b69] x = mpz(8697650338018556857196672168418493936492589454685953226387800802148791173126201704559664294438994443312628470960093370399427907591481958482977688876871208), m = mpz(8206912679750924871060807948577706910431377366961731262523431355550191424970474321210924371939645867568323618900657590444152383410800277507168351971439484)
[TRACE] [STSUser] [c80d165b-9cda-4148-b3bb-273f62780b57] y = mpz(5245509192338740936061774283897612843695693312366044458117615590255684062858665539971191794507637552562058073231285296902563234607074273724182008980402050), m = mpz(3978297874747629805963967648858328695983325581983291554173436967437455932158998533331003567112082558015186718250854005041060198285912728397032713319673827)
[TRACE] [STSUser] [41a93646-3863-4735-9be3-fbef365c1b69] derived key: a947566aee7d5ed92ba11b82c1207f60018438e269f6bbf56aca0f5d60bc9b49
[TRACE] [STSUser] [41a93646-3863-4735-9be3-fbef365c1b69] derived key: a947566aee7d5ed92ba11b8

In [25]:
# Теперь врага создам
mallory = STSUser(params, fake_data=alice.public_key)

[TRACE] [STSUser] client e459d466-e470-47b0-81f0-794a9c93b599 created. PK = Public RSA key at 0x104697400, SK = Private RSA key at 0x10468FF70


In [26]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [STSUser] [e459d466-e470-47b0-81f0-794a9c93b599] x = mpz(7728577286096457007368142787491415928484827747233002859963135771052475586567208565945865965679524194612861232273529659305926527711330499640546045812413922), m = mpz(4980717061630343319363175730755942954150697164627171938029348862527080355908560520096896163944081520188291079251584410792710470708338765112369222791290532)
[TRACE] [STSUser] [c80d165b-9cda-4148-b3bb-273f62780b57] y = mpz(5141030392929874736229039942742942543271630585462280158333033958163322200356872883511869774176265073843528241935539364404947075052479823494376131152195416), m = mpz(1902330918547631424198257636689156045514835198774429258203528546454630865858752641489667922229532706719496174258933756545913960929514247935546890561517625)
[TRACE] [STSUser] [e459d466-e470-47b0-81f0-794a9c93b599] Invalid signature
[TRACE] [STSUser] [e459d466-e470-47b0-81f0-794a9c93b599] Another user refused key exchange


In [27]:
bob.initiate_key_exchange(mallory)

[TRACE] [STSUser] [c80d165b-9cda-4148-b3bb-273f62780b57] x = mpz(7241009918880463587752109102411120159577863465841309120818384801220245292111106986898256167507970199309292615305410327089154123194335745657535976102520749), m = mpz(7008717753999639024426227058075921910856307824634422532887217543686816180037197723478689868481366067649200328277327274678099407132771487665218835159737049)
[TRACE] [STSUser] [e459d466-e470-47b0-81f0-794a9c93b599] y = mpz(521586077876688369032363629655128225765072592182465748575966471151139103928814819176401308143417409176994896462369856284266459999937589983368772575167499), m = mpz(9615179367177951031331220679399183443970609493995843962533260803098530835059305954094152183452746449818183297455578433790869556945898019728762923157314571)
[TRACE] [STSUser] [c80d165b-9cda-4148-b3bb-273f62780b57] Invalid signature
